<a href="https://colab.research.google.com/github/banhsbao/mse_python_chatbox/blob/main/Chatbot_LLM_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get clean
!sudo apt-get update
!sudo apt-get install poppler-utils

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq openai==0.27.4 --progress-bar off
!pip install -Uqqq watermark==2.3.1 --progress-bar off
!pip install -qqq langchain==0.0.173 --progress-bar off
!pip install -qqq chromadb==0.3.23 --progress-bar off
!pip install -qqq pypdf==3.8.1 --progress-bar off
!pip install -qqq pygpt4all==1.1.0 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install -Uqqq tiktoken==0.3.3 --progress-bar off
!pip install -Uqqq unstructured[local-inference]==0.5.12 --progress-bar off

In [ ]:
import os
import textwrap

import chromadb
import langchain
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from pdf2image import convert_from_path

Download file pdf

In [ ]:
# Download file pdf
!gdown https://drive.google.com/uc?id=16kk1KEqoorpVghSlMLI7O7PCfhr0Fucc

Downloading...
From: https://drive.google.com/uc?id=16kk1KEqoorpVghSlMLI7O7PCfhr0Fucc
To: /content/Li_thuyet_Hadoop.pdf
100% 392k/392k [00:00<00:00, 120MB/s]


Load Data & Model

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [ ]:
model = OpenAI(temperature=0, model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
images = convert_from_path("Li_thuyet_Hadoop.pdf", dpi=88)
len(images)

2

In [ ]:
images[0]

Use UnstructuredPDFLoader to load PDFs

In [ ]:
# Use UnstructuredPDFLoader to load PDFs from the Internets
pdf_loader = UnstructuredPDFLoader("Li_thuyet_Hadoop.pdf")
pdf_pages = pdf_loader.load_and_split()

In [ ]:
# Text Splitters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(pdf_pages)
len(texts)

2

Create Embeddings & Vectorstores

In [ ]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [ ]:
db = Chroma.from_documents(texts, hf_embeddings, persist_directory="db")

#Use a Chain

In [ ]:
custom_prompt_template = """Sử dụng các thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời, chỉ cần nói rằng bạn không biết, đừng cố bịa ra câu trả lời.
Tất cả câu trả lời của bạn đều phải trả lời bằng tiếng việt

Context: {context}
Question: {question}

"""

In [ ]:
from langchain import PromptTemplate
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [ ]:
prompt = set_custom_prompt()
chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    chain_type_kwargs={'prompt': prompt}
)

#QA Chatbot

In [ ]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [ ]:
query = "Phạm vi áp dụng có nằm trong mục đích hay không?"
response = chain.run(query)
print_response(response)

Có, phạm vi áp dụng nằm trong mục đích của quy trình phối hợp tiếp nhận triển khai yêu cầu xây
dựng/nâng cấp bài toán AI của Tổng Công ty Cổ phần Công trình Viettel (VCC).


In [ ]:
query = "MapReduce là gì?"
response = chain.run(query)
print_response(response)

MapReduce là một mô hình lập trình và kỹ thuật xử lý dữ liệu phân tán trong Big Data. Nó bao gồm hai
tác vụ quan trọng là map và reduce. Mô hình này được phát triển bởi Google và được sử dụng để tự
động song song hóa và phân phối các phép tính quy mô lớn trên các cụm máy tính lớn.


In [ ]:
query = "Hadoop là gì?"
response = chain.run(query)
print_response(response)

Hadoop là một hệ sinh thái phần mềm mã nguồn mở được sử dụng để xử lý và lưu trữ dữ liệu lớn. Nó bao
gồm các thành phần như HDFS, Hive, Pig, YARN, MapReduce, Spark, HBase, Oozie, Sqoop, Zookeeper, và
nhiều công cụ khác. Hadoop được xây dựng trên ngôn ngữ Java và tương thích trên nhiều nền tảng khác
nhau.


In [ ]:
query = "Kiến trúc của Hadoop gồm những gì?"
response = chain.run(query)
print_response(response)

Kiến trúc của Hadoop bao gồm các thành phần sau: 1. Hadoop Distributed File System (HDFS): Hệ thống
file phân tán của Hadoop, nằm ở trên cùng của hệ thống file cục bộ và giám sát quá trình. 2.
MapReduce: Mô hình xử lý dữ liệu phân tán trong Hadoop, sử dụng các công cụ để phân tách và xử lý dữ
liệu trên các nút trong cụm. 3. YARN (Yet Another Resource Negotiator): Quản lý các thành viên trong
nhóm máy chủ và phân phối tài nguyên cho các ứng dụng chạy trên Hadoop. 4. ZooKeeper: Hệ thống quản
lý và bầu cử leader trong Hadoop, đảm bảo tính nhất quán và sẵn sàng của dữ liệu. 5. Hadoop Common:
Bao gồm các thư viện và công cụ chung được sử dụng bởi các thành phần khác trong Hadoop. 6. Hadoop
Ozone: Hệ thống lưu trữ đối tượng phân tán trong Hadoop, cung cấp khả năng lưu trữ và truy xuất dữ
liệu theo kiểu đối tượng. 7. Hadoop HDFS Federation: Cung cấp khả năng mở rộng và phân chia dữ liệu
trên nhiều cụm HDFS. 8. Hadoop HDFS High Availability: Cung cấp khả năng sao lưu và khôi phục tự
động cho H